In [1]:
import warnings
from importlib import reload
from functools import partial, lru_cache
import itertools
from time import monotonic

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, svm, metrics
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm, trange
from joblib import delayed
from modAL import batch
from art.metrics import empirical_robustness
from art.attacks.evasion import *
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from tabulate import tabulate
from art.estimators.classification.scikitlearn import ScikitlearnSVC
from sklearn.metrics.pairwise import paired_distances, euclidean_distances
import scipy
from tvregdiff.tvregdiff import TVRegDiff
from tabulate import tabulate

import ipywidgets as widgets
from ipywidgets import interact

import libactive
import libadversarial
import libstop
from libactive import MyActiveLearner, active_split
from libadversarial import adversarial, uncertainty, random_batch, uncertainty_stop
from libutil import ProgressParallel
from libdatasets import *
import librun
from librun import run

In [2]:
import libdatasets; reload(libdatasets); from libdatasets import *

In [3]:
matrix = {
    # Dataset fetchers should cache if possible
    # Lambda wrapper required for function to be pickleable (sent to other threads via joblib)
    "datasets": [
        ("newsgroups", wrap(newsgroups, None)),
        ("rcv1", wrap(rcv1, None)),
        ("webkb", wrap(webkb, None)),
        ("spamassassin", wrap(spamassassin, None)),
        ("avila", wrap(avila, None)),
        ("smartphone", wrap(smartphone, None)),
        ("swarm", wrap(swarm, None)),
        ("sensorless", wrap(sensorless, None)),
        ("splice", wrap(splice, None)),
        ("anuran", wrap(anuran, None)),        
    ],
    "dataset_mutators": {
        "none": (lambda *x, **kwargs: x),
    },
    "methods": [
        ("uncertainty", partial(uncertainty_stop, n_instances=10)),
    ],
    "models": [
        "svm-linear"
    ],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 10,
        "test_size": {
            "newsgroups_faith": 500,
            "newsgroups_graphics": 500,
            "newsgroups_hardware": 500,
            "newsgroups_sports_crypto": 500,
            "*": 0.5
        },
        "n_runs": 10,
        "ret_classifiers": True,
        "ensure_y": True,
        "stop_info": True,
        "aggregate": False,
        "stop_function": ("len1000", lambda learner: learner.y_training.shape[0] >= 1000),
        "pool_subsample": 1000
    }
}


In [4]:
reload(libactive); from libactive import active_split

In [12]:
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from time import monotonic
import nesi_bias
reload(nesi_bias)
reload(libactive); from libactive import active_split
from nesi_bias import bias
from dotenv import load_dotenv; load_dotenv()
results_nn = []
for (name, dataset) in matrix['datasets']:
    print(f"{name}")
    X, y = dataset()

    if isinstance(X, scipy.sparse.csr_matrix):
        results_nn.append([name, 0,0,0])
        continue

    print("  Start split")
    X_labelled, X_unlabelled, Y_labelled, Y_oracle, X_test, y_test = active_split(
        *dataset(), mutator=partial(bias, amount=0.5), test_size=0.5, labeled_size=10, shuffle=True, random_state=np.random
    )
    print("  End split")

    if isinstance(X_labelled, scipy.sparse.csr_matrix):
        X = scipy.sparse.vstack((X_labelled, X_unlabelled))
    else:
        X = np.concatenate((X_labelled, X_unlabelled))
    y = np.concatenate((Y_labelled, Y_oracle))

    
    clf = SVC(kernel='linear', probability=True)

    y_short = y[:10]
    X_short = X[:10]
    for klass in np.unique(y):
        if klass not in y_short:
            idx = np.where(y==klass)[0][0]
            y_short = np.concatenate((y_short, [y[idx]]), axis=0)
            if isinstance(X_short, scipy.sparse.csr_matrix):
                X_short = scipy.sparse.vstack((
                    X_short, 
                    X[idx]
                ))
            else:
                X_short = np.concatenate((X_short, [X[idx]]), axis=0)
    #print(np.unique(y_short))

    print("  Start short fit")
    clf.fit(X_short, y_short)
    print("  Stop short fit")
    start = clf.score(X_test, y_test)
    start_t = monotonic()
    print("  Start long fit")
    clf.fit(X, y)
    print("  Stop long fit")
    time = monotonic() - start_t
    final = clf.score(X_test, y_test)
    results_nn.append([name, start, final, time])

newsgroups
rcv1
webkb
spamassassin
avila
  Start split
[    0     1     3 ... 10428 10430 10432]
unique ['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'W' 'X' 'Y']
oracle (array(['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'W', 'X', 'Y'],
      dtype=object), array([ 925,   33, 5303, 5640,  536,   37,  184,   32,   17,  186,  114],
      dtype=int64))


C:\Users\Zac\Programming\python\research\libactive.py:74: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if klass not in Y_labelled:


IndexError: index 0 is out of bounds for axis 0 with size 0